In [59]:
import psycopg2
import numpy as np
import pandas as pd

In [60]:
conn = psycopg2.connect(host="127.0.0.1", port=5432,user="postgres",password="123456", database='aidb')
cur = conn.cursor()

In [61]:
def to_vals(data_list):
    for dat in data_list:
        val = dat[0]
        if val is not None: break
    try:
        float(val)
        return np.array(data_list, dtype=float).squeeze()
    except:
#         print(val)
        res = []
        for dat in data_list:
            try:
                mi = dat[0].timestamp()
            except:
                mi = 0
            res.append(mi)
        return np.array(res)

In [62]:
forest_schema = {'forest': ['forest.Hillshade_Noon', 'forest.Hillshade_3pm', 'forest.Horizontal_Distance_To_Hydrology', 'forest.Aspect', 'forest.Horizontal_Distance_To_Roadways', 'forest.Elevation', 'forest.Horizontal_Distance_To_Fire_Points', 'forest.Hillshade_9am', 'forest.Vertical_Distance_To_Hydrology', 'forest.Slope']}

In [63]:
hist_file = pd.DataFrame(columns=['table','column','bins','table_column'])
hist_file

,table,column,bins,table_column


In [65]:
for table, columns in forest_schema.items():
    for column in columns:
        cmd = 'select {} from {}'.format(column, table)
        cur.execute(cmd)
        col = cur.fetchall()
        col_array = to_vals(col)
        hists = np.nanpercentile(col_array, range(0, 101, 2), axis=0)
        for h in hists:
            print(h)
        res_dict = {
            'table':table,
            'column':column,
            'table_column': '.'.join((table, column)),
            'bins':hists
        }
        hist_file = hist_file.append(res_dict,ignore_index=True)

0.0


KeyboardInterrupt: 

In [14]:
hist_file

,table,column,bins,table_column
0,forest,forest.Hillshade_Noon,"[0.0, 172.0, 183.0, 189.0, 194.0, 198.0, 201.0...",forest.forest.Hillshade_Noon
1,forest,forest.Hillshade_3pm,"[0.0, 56.0, 72.0, 82.0, 89.0, 95.0, 99.0, 103....",forest.forest.Hillshade_3pm
2,forest,forest.Horizontal_Distance_To_Hydrology,"[0.0, 0.0, 0.0, 30.0, 30.0, 30.0, 42.0, 60.0, ...",forest.forest.Horizontal_Distance_To_Hydrology
3,forest,forest.Aspect,"[0.0, 4.0, 9.0, 14.0, 19.0, 24.0, 29.0, 33.0, ...",forest.forest.Aspect
4,forest,forest.Horizontal_Distance_To_Roadways,"[0.0, 212.0, 330.0, 424.0, 510.0, 591.0, 663.0...",forest.forest.Horizontal_Distance_To_Roadways
5,forest,forest.Elevation,"[1859.0, 2227.0, 2356.0, 2452.0, 2530.0, 2581....",forest.forest.Elevation
6,forest,forest.Horizontal_Distance_To_Fire_Points,"[0.0, 258.0, 366.0, 454.0, 531.0, 595.0, 658.0...",forest.forest.Horizontal_Distance_To_Fire_Points
7,forest,forest.Hillshade_9am,"[0.0, 140.0, 155.0, 164.0, 171.0, 176.0, 181.0...",forest.forest.Hillshade_9am
8,forest,forest.Vertical_Distance_To_Hydrology,"[-173.0, -26.0, -11.0, -6.0, -2.0, 0.0, 0.0, 0...",forest.forest.Vertical_Distance_To_Hydrology
9,forest,forest.Slope,"[0.0, 2.0, 3.0, 4.0, 5.0, 5.0, 6.0, 6.0, 7.0, ...",forest.forest.Slope


In [18]:
print(type(hist_file))

<class 'pandas.core.frame.DataFrame'>


In [19]:
hist_file['bins']

0    [0.0, 172.0, 183.0, 189.0, 194.0, 198.0, 201.0...
1    [0.0, 56.0, 72.0, 82.0, 89.0, 95.0, 99.0, 103....
2    [0.0, 0.0, 0.0, 30.0, 30.0, 30.0, 42.0, 60.0, ...
3    [0.0, 4.0, 9.0, 14.0, 19.0, 24.0, 29.0, 33.0, ...
4    [0.0, 212.0, 330.0, 424.0, 510.0, 591.0, 663.0...
5    [1859.0, 2227.0, 2356.0, 2452.0, 2530.0, 2581....
6    [0.0, 258.0, 366.0, 454.0, 531.0, 595.0, 658.0...
7    [0.0, 140.0, 155.0, 164.0, 171.0, 176.0, 181.0...
8    [-173.0, -26.0, -11.0, -6.0, -2.0, 0.0, 0.0, 0...
9    [0.0, 2.0, 3.0, 4.0, 5.0, 5.0, 6.0, 6.0, 7.0, ...
Name: bins, dtype: object

In [24]:
outputpath = "D:\CECode\importForestAndPower\\histogram_forest.csv"
hist_file.to_csv(outputpath,sep=',',index=False,header=False)

In [52]:
for column, bins in zip(hist_file['column'], hist_file['bins']):
    print(column, bins)

forest.Hillshade_Noon [  0. 172. 183. 189. 194. 198. 201. 203. 206. 207. 209. 211. 212. 214.
 215. 216. 217. 218. 219. 220. 221. 222. 223. 224. 225. 226. 227. 228.
 229. 229. 230. 231. 232. 233. 234. 235. 236. 237. 238. 239. 240. 241.
 243. 244. 245. 247. 248. 250. 251. 253. 254.]
forest.Hillshade_3pm [  0.  56.  72.  82.  89.  95.  99. 103. 107. 110. 113. 116. 118. 120.
 122. 125. 127. 129. 130. 132. 134. 136. 138. 139. 141. 143. 145. 146.
 148. 150. 152. 154. 156. 158. 160. 162. 165. 167. 170. 172. 175. 178.
 181. 184. 187. 191. 196. 201. 208. 218. 254.]
forest.Horizontal_Distance_To_Hydrology [   0.    0.    0.   30.   30.   30.   42.   60.   67.   67.   85.   90.
   95.  108.  120.  124.  134.  150.  150.  162.  175.  182.  192.  201.
  212.  218.  234.  242.  256.  268.  277.  295.  300.  319.  330.  342.
  361.  376.  391.  408.  430.  451.  474.  503.  531.  564.  601.  655.
  721.  834. 1397.]
forest.Aspect [  0.   4.   9.  14.  19.  24.  29.  33.  38.  43.  47.  51.  56.  61.
